In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report

In [ ]:
path = "Proyecto2/Processed_Dataframes/"

Accidentes24_data = pd.read_excel("Proyecto2/Processed_Dataframes/Accidentes24_data.xlsx", index_col=None)
Personas24_data = pd.read_excel("Proyecto2/Processed_Dataframes/Personas24_data.xlsx", index_col=None)

Accidentes25_data = pd.read_excel("Proyecto2/Processed_Dataframes/Accidentes25_data.xlsx", index_col=None)
Personas25_data = pd.read_excel("Proyecto2/Processed_Dataframes/Personas25_data.xlsx", index_col=None)

In [64]:
Accidentes24_data_rev = Accidentes24_data.copy()
Personas24_data_rev = Personas24_data.copy()
Accidentes25_data_rev = Accidentes25_data.copy()
Personas25_data_rev = Personas25_data.copy()

Accidentes24_data_rev['Hora'] = pd.to_datetime(Accidentes24_data_rev['Hora'], errors='coerce')
Accidentes25_data_rev['HORA'] = pd.to_datetime(Accidentes25_data_rev['HORA'], errors='coerce')

Personas24_data_rev['Hora'] = pd.to_datetime(Personas24_data_rev['Hora'], errors='coerce')
Personas25_data_rev['HORA'] = pd.to_datetime(Personas25_data_rev['HORA'], errors='coerce')

def normalizar_columna(col):
    col = col.strip().lower()
    col = col.replace('á', 'a')
    col = col.replace('é', 'e')
    col = col.replace('í', 'i')
    col = col.replace('ó', 'o')
    col = col.replace('ú', 'u')
    col = col.replace('ñ', 'ñ')
    col = col.replace('ü', 'u')
    return col

Accidentes24_data_rev['Región'] = Accidentes24_data_rev['Región'].apply(normalizar_columna)
Accidentes24_data_rev['Urbano/Rural'] = Accidentes24_data_rev['Urbano/Rural'].apply(normalizar_columna)
Accidentes24_data_rev['Nomcomuna'] = Accidentes24_data_rev['Nomcomuna'].apply(normalizar_columna)
Personas24_data_rev['Región'] = Personas24_data_rev['Región'].apply(normalizar_columna)

C:\Users\ivans\AppData\Local\Temp\ipykernel_15768\2708195555.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  Accidentes24_data_rev['Hora'] = pd.to_datetime(Accidentes24_data_rev['Hora'], errors='coerce')
C:\Users\ivans\AppData\Local\Temp\ipykernel_15768\2708195555.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  Accidentes25_data_rev['HORA'] = pd.to_datetime(Accidentes25_data_rev['HORA'], errors='coerce')
C:\Users\ivans\AppData\Local\Temp\ipykernel_15768\2708195555.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  Personas24_data_rev['Hora'] = pd.to_datetime(Person

In [65]:

  #2024

  print(f"Dataframe Acc24: columnas -> {Accidentes24_data_rev.columns.tolist()}")

  print("-------------------------------------------------------------------")

  print(f"Dataframe Acc25: columnas -> {Accidentes25_data_rev.columns.tolist()}")

  print("-------------------------------------------------------------------")

  print(f"Dataframe Perso24: columnas -> {Personas24_data_rev.columns.tolist()}")

  print("-------------------------------------------------------------------")

  print(f"Dataframe Perso35: columnas -> {Personas25_data_rev.columns.tolist()}")

Dataframe Acc24: columnas -> ['Idaccidente', 'Fecha', 'Mes', 'Hora', 'Comuna', 'Nomcomuna', 'Región', 'Accdtes.', 'Causas', 'Urbano/Rural', 'Calleuno', 'Calledos', 'Frentenumero', 'Ruta', 'Ubicación km', 'Muertos', 'Graves', 'M/Grave', 'Leves', 'Ilesos', 'Parte Nro.', 'Tribunal']
-------------------------------------------------------------------
Dataframe Acc25: columnas -> ['ID', 'FECHA', 'MES', 'HORA', 'COMUNA', 'ZONA', 'TIPO', 'CAUSA', 'SECTOR', 'CALLE_1', 'CALLE_2', 'KM', 'FALLECIDO', 'GRAVE', 'M/GRAVE', 'LEVE', 'ILESO', 'PARTE NRO.', 'TRIBUNAL']
-------------------------------------------------------------------
Dataframe Perso24: columnas -> ['Idaccidente', 'Fecha', 'Mes', 'Hora', 'Comuna', 'Nomcomuna', 'Región', 'Calidad', 'Sexo', 'Edad', 'Resultado']
-------------------------------------------------------------------
Dataframe Perso35: columnas -> ['ID', 'FECHA', 'MES', 'HORA', 'COMUNA', 'ZONA', 'CALIDAD', 'SEXO', 'EDAD', 'RESULTADO']


In [66]:
Regiones_Comunas = pd.read_excel("Proyecto2/Utilities/regiones_comunas.xlsx", index_col=None)
Regiones_Comunas_Parche = pd.DataFrame({
    'Nombre_Comuna': ['Camiña', 'Colchane', 'Huara', 'Pica', 'Pozo Almonte'],
    'Codigo_Comuna': [1102, 1103, 1104, 1105, 1106],
    'Nombre_Region': ['De Tarapacá', 'De Tarapacá', 'De Tarapacá', 'De Tarapacá', 'De Tarapacá'],
    'Codigo_Region': [1, 1, 1, 1, 1]
})
Regiones_Comunas = pd.concat([Regiones_Comunas, Regiones_Comunas_Parche], ignore_index=True)

In [67]:
mapa_comuna = dict(zip(Regiones_Comunas['Codigo_Comuna'], Regiones_Comunas['Nombre_Comuna']))
mapa_region = dict(zip(Regiones_Comunas['Codigo_Comuna'], Regiones_Comunas['Nombre_Region']))

In [68]:
Accidentes24_data_rev['Nomcomuna'] = Accidentes24_data_rev['Comuna'].map(mapa_comuna)
Accidentes24_data_rev['Región'] = Accidentes24_data_rev['Comuna'].map(mapa_region)

In [69]:
Accidentes25_data_rev['COMUNA'] = Accidentes25_data_rev['COMUNA'].apply(normalizar_columna)
Accidentes24_data_rev['Nomcomuna'] = Accidentes24_data_rev['Nomcomuna'].apply(normalizar_columna)

In [70]:
def clasificar_ubicacion(row):
    if pd.notna(row['Ruta']):
        return "Ruta"
    elif pd.notna(row['Calleuno']) and pd.notna(row['Calledos']):
        return "Intersección"
    elif pd.notna(row['Calleuno']):
        return "Una calle"
    else:
        return "Desconocido"  
Accidentes24_data_rev['TipoUbicacion'] = Accidentes24_data_rev.apply(clasificar_ubicacion, axis=1)

In [71]:
Accidentes25_data_rev.columns
renombrar_acc25 = {
    'ID': 'Idaccidente',
    'FECHA': 'Fecha',
    'MES': 'Mes',
    'HORA': 'Hora',
    'COMUNA': 'Nomcomuna',
    'ZONA': 'Región',
    'TIPO': 'Accdtes.',
    'CAUSA': 'Causas',
    'SECTOR': 'Urbano/Rural',
    'CALLE_1': 'Calleuno',
    'CALLE_2': 'Calledos',
    'KM': 'Ubicación km',
    'FALLECIDO': 'Muertos',
    'GRAVE': 'Graves',
    'M/GRAVE': 'M/Grave',
    'LEVE': 'Leves',
    'ILESO': 'Ilesos',
    'PARTE NRO.': 'Parte Nro.',
    'TRIBUNAL': 'Tribunal'
}
Accidentes25_data_rev = Accidentes25_data_rev.rename(columns=renombrar_acc25)


In [72]:
def clasificar_ubicacion_2025(row):
    calle1 = str(row['Calleuno']).strip().upper() if pd.notna(row['Calleuno']) else ''
    calle2 = str(row['Calledos']).strip() if pd.notna(row['Calledos']) else ''

    if calle1 and calle2:
        return "Intersección"
    elif calle1.startswith("RUTA"):
        return "Ruta"
    elif calle1:
        return "Una calle"
    else:
        return "Desconocido"

Accidentes25_data_rev['TipoUbicacion'] = Accidentes25_data_rev.apply(clasificar_ubicacion_2025, axis=1)

In [73]:
Accidentes24_data_rev.head()
Accidentes24_data_rev['Accdtes.'] = Accidentes24_data_rev['Accdtes.'].apply(normalizar_columna)
Accidentes24_data_rev['Mes'] = Accidentes24_data_rev['Mes'].apply(normalizar_columna)

In [74]:
Accidentes25_data_rev.head()
Accidentes25_data_rev['Mes'] = Accidentes25_data_rev['Mes'].apply(normalizar_columna)
Accidentes25_data_rev['Accdtes.'] = Accidentes25_data_rev['Accdtes.'].apply(normalizar_columna)
Accidentes25_data_rev['Urbano/Rural'] = Accidentes25_data_rev['Urbano/Rural'].apply(normalizar_columna)

In [75]:
zona_a_region = {
    'ZONA METROPOLITANA': 'Metropolitana de Santiago',
    'ZONA DEL BIOBIO': 'Del Bíobío',
    'ZONA DEL MAULE': 'Del Maule',
    'ZONA DE VALPARAISO': 'De Valparaíso',
    'ZONA DE LA ARAUCANIA': 'De La Araucanía',
    'ZONA DE LOS LAGOS': 'De Los Lagos',
    'ZONA DEL LIB. B. OHIGGINS': "Del Libertador B. O'Higgins",
    'ZONA DE COQUIMBO': 'De Coquimbo',
    'ZONA DE ANTOFAGASTA': 'De Antofagasta',
    'ZONA DE NUBLE': 'De Ñuble',
    'ZONA DE ARICA Y PARINACOTA': 'De Arica y Parinacota',
    'ZONA DE TARAPACA': 'De Tarapacá',
    'ZONA DE LOS RIOS': 'De Los Ríos',
    'ZONA DE ATACAMA': 'De Atacama',
    'ZONA DE MAGALLANES': 'De Magallanes y de La Antártica Chilena',
    'ZONA DE AYSEN': 'De Aisén del Gral. C. Ibáñez del Campo'
}
Accidentes25_data_rev['Región'] = Accidentes25_data_rev['Región'].map(zona_a_region)

In [76]:
Accidentes24_dt = Accidentes24_data_rev.copy()
Accidentes25_dt = Accidentes25_data_rev.copy()

In [77]:
Accidentes25_dt.columns

Index(['Idaccidente', 'Fecha', 'Mes', 'Hora', 'Nomcomuna', 'Región',
       'Accdtes.', 'Causas', 'Urbano/Rural', 'Calleuno', 'Calledos',
       'Ubicación km', 'Muertos', 'Graves', 'M/Grave', 'Leves', 'Ilesos',
       'Parte Nro.', 'Tribunal', 'TipoUbicacion'],
      dtype='object')

In [78]:
Accidentes24_dt.columns

Index(['Idaccidente', 'Fecha', 'Mes', 'Hora', 'Comuna', 'Nomcomuna', 'Región',
       'Accdtes.', 'Causas', 'Urbano/Rural', 'Calleuno', 'Calledos',
       'Frentenumero', 'Ruta', 'Ubicación km', 'Muertos', 'Graves', 'M/Grave',
       'Leves', 'Ilesos', 'Parte Nro.', 'Tribunal', 'TipoUbicacion'],
      dtype='object')

In [79]:
Accidentes24_dt = Accidentes24_dt.drop(columns=['Comuna', 'Calleuno', 'Calledos', 'Frentenumero', 'Ruta', 'Ubicación km', 'Parte Nro.', 'Tribunal'])
Accidentes25_dt = Accidentes25_dt.drop(columns=['Calleuno', 'Calledos', 'Ubicación km', 'Parte Nro.', 'Tribunal'])

In [80]:
Accidentes24_dt.head()

,Idaccidente,Fecha,Mes,Hora,Nomcomuna,Región,Accdtes.,Causas,Urbano/Rural,Muertos,Graves,M/Grave,Leves,Ilesos,TipoUbicacion
0,1586623,2024-01-01,enero,2025-06-23 00:00:00,quirihue,De Ñuble,colision,Conducción no atento condiciones transito momento,urbano,0,0,0,3,0,Intersección
1,1583417,2024-01-01,enero,2025-06-23 00:30:00,san felipe,De Valparaíso,choque,Conducción no atento condiciones transito momento,urbano,0,0,0,0,2,Intersección
2,1584272,2024-01-01,enero,2025-06-23 00:30:00,ñiquen,De Ñuble,choque,Animales sueltos via publica,rural,0,0,0,0,1,Ruta
3,1581924,2024-01-01,enero,2025-06-23 00:45:00,puente alto,Metropolitana de Santiago,colision,Conducción no atento condiciones transito momento,urbano,0,1,0,0,1,Intersección
4,1581883,2024-01-01,enero,2025-06-23 01:00:00,talca,Del Maule,colision,Conducción por izquierda eje calzada,urbano,0,0,0,0,2,Intersección


In [81]:
Accidentes25_dt.head()

,Idaccidente,Fecha,Mes,Hora,Nomcomuna,Región,Accdtes.,Causas,Urbano/Rural,Muertos,Graves,M/Grave,Leves,Ilesos,TipoUbicacion
0,5660,2025-01-01,enero,2025-06-23 00:01:00,colina,Metropolitana de Santiago,choque,"ADELANTAMIENTO EN CRUCE, CURVA, CUESTA, PUENTE",rural,0,0,0,1,0,Ruta
1,3703,2025-01-01,enero,2025-06-23 00:01:00,renca,Metropolitana de Santiago,choque,CONDUCTOR NO ATENTO CONDICIONES TRANSITO MOMENTO,urbano,0,0,0,0,1,Una calle
2,5995,2025-01-01,enero,2025-06-23 00:01:00,mostazal,Del Libertador B. O'Higgins,choque,CONDUCTOR EN ESTADO DE EBRIEDAD,urbano,0,0,0,0,1,Intersección
3,3103,2025-01-01,enero,2025-06-23 01:01:00,maule,Del Maule,impacto con animal,ANIMALES SUELTOS VIA PUBLICA,rural,0,0,0,1,0,Ruta
4,3192,2025-01-01,enero,2025-06-23 01:01:00,valparaiso,De Valparaíso,colision lateral,ADELANTAMIENTO SIN EL ESPACIO,urbano,0,0,0,0,2,Intersección


In [82]:
accidentes_training = pd.concat([Accidentes24_dt, Accidentes25_dt], ignore_index=True)
accidentes_training.head()

,Idaccidente,Fecha,Mes,Hora,Nomcomuna,Región,Accdtes.,Causas,Urbano/Rural,Muertos,Graves,M/Grave,Leves,Ilesos,TipoUbicacion
0,1586623,2024-01-01,enero,2025-06-23 00:00:00,quirihue,De Ñuble,colision,Conducción no atento condiciones transito momento,urbano,0,0,0,3,0,Intersección
1,1583417,2024-01-01,enero,2025-06-23 00:30:00,san felipe,De Valparaíso,choque,Conducción no atento condiciones transito momento,urbano,0,0,0,0,2,Intersección
2,1584272,2024-01-01,enero,2025-06-23 00:30:00,ñiquen,De Ñuble,choque,Animales sueltos via publica,rural,0,0,0,0,1,Ruta
3,1581924,2024-01-01,enero,2025-06-23 00:45:00,puente alto,Metropolitana de Santiago,colision,Conducción no atento condiciones transito momento,urbano,0,1,0,0,1,Intersección
4,1581883,2024-01-01,enero,2025-06-23 01:00:00,talca,Del Maule,colision,Conducción por izquierda eje calzada,urbano,0,0,0,0,2,Intersección


In [83]:
Personas24_data_rev = Personas24_data.copy()
Personas25_data_rev = Personas25_data.copy()
Personas24_data_rev.columns
Personas24_data_rev['Nomcomuna'] = Personas24_data_rev['Comuna'].map(mapa_comuna)
Personas24_data_rev['Región'] = Personas24_data_rev['Comuna'].map(mapa_region)

In [84]:
Personas24_data_rev['Mes'] = Personas24_data_rev['Mes'].apply(normalizar_columna)
Personas24_data_rev['Nomcomuna'] = Personas24_data_rev['Nomcomuna'].apply(normalizar_columna)
Personas24_data_rev['Calidad'] = Personas24_data_rev['Calidad'].apply(normalizar_columna)
Personas24_data_rev['Sexo'] = Personas24_data_rev['Sexo'].apply(normalizar_columna)
Personas24_data_rev['Resultado'] = Personas24_data_rev['Resultado'].apply(normalizar_columna)
Personas24_data_rev['Edad'].fillna(0, inplace=True)
Personas24_data_rev['Sexo'].fillna('desconocido', inplace=True)

Personas24_data_rev.head()

C:\Users\ivans\AppData\Local\Temp\ipykernel_15768\3463182794.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  Personas24_data_rev['Edad'].fillna(0, inplace=True)
C:\Users\ivans\AppData\Local\Temp\ipykernel_15768\3463182794.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For 

,Idaccidente,Fecha,Mes,Hora,Comuna,Nomcomuna,Región,Calidad,Sexo,Edad,Resultado
0,1586623,2024-01-01,enero,00:00:00,8414,quirihue,De Ñuble,conductor,masculino,40.0,leve
1,1586623,2024-01-01,enero,00:00:00,8414,quirihue,De Ñuble,pasajero,femenino,24.0,leve
2,1586623,2024-01-01,enero,00:00:00,8414,quirihue,De Ñuble,conductor,masculino,23.0,leve
3,1583417,2024-01-01,enero,00:30:00,5701,san felipe,De Valparaíso,conductor,masculino,35.0,ileso
4,1583417,2024-01-01,enero,00:30:00,5701,san felipe,De Valparaíso,conductor,masculino,0.0,ileso


In [85]:
renombrar_personas = {
    'ID': 'Idaccidente',
    'FECHA': 'Fecha',
    'MES': 'Mes',
    'HORA': 'Hora',
    'COMUNA': 'Nomcomuna',
    'ZONA': 'Región',
    'CALIDAD': 'Calidad',
    'SEXO': 'Sexo',
    'EDAD': 'Edad',
    'RESULTADO': 'Resultado'
}

Personas25_data_rev = Personas25_data_rev.rename(columns=renombrar_personas)
Personas25_data_rev['Edad'].fillna(0, inplace=True)
Personas25_data_rev['Sexo'].fillna('desconocido', inplace=True)
Personas25_data_rev['Mes'] = Personas25_data_rev['Mes'].apply(normalizar_columna)
Personas25_data_rev['Nomcomuna'] = Personas25_data_rev['Nomcomuna'].apply(normalizar_columna)
Personas25_data_rev['Calidad'] = Personas25_data_rev['Calidad'].apply(normalizar_columna)
Personas25_data_rev['Resultado'] = Personas25_data_rev['Resultado'].apply(normalizar_columna)
Personas25_data_rev['Sexo'] = Personas25_data_rev['Sexo'].apply(normalizar_columna)
Personas25_data_rev['Región'] = Personas25_data_rev['Región'].map(zona_a_region)
Personas25_data_rev.head()

C:\Users\ivans\AppData\Local\Temp\ipykernel_15768\2487853267.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  Personas25_data_rev['Edad'].fillna(0, inplace=True)
C:\Users\ivans\AppData\Local\Temp\ipykernel_15768\2487853267.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

Fo

,Idaccidente,Fecha,Mes,Hora,Nomcomuna,Región,Calidad,Sexo,Edad,Resultado
0,3703,2025-01-01,enero,00:01,renca,Metropolitana de Santiago,se da a la fuga,masculino,0.0,ileso
1,5995,2025-01-01,enero,00:01,mostazal,Del Libertador B. O'Higgins,conductor,masculino,16.0,ileso
2,5660,2025-01-01,enero,00:01,colina,Metropolitana de Santiago,conductor,masculino,46.0,leve
3,3192,2025-01-01,enero,01:01,valparaiso,De Valparaíso,conductor,masculino,31.0,ileso
4,3192,2025-01-01,enero,01:01,valparaiso,De Valparaíso,conductor,masculino,29.0,ileso


In [86]:
Personas24_dt = Personas24_data_rev.copy()
Personas25_dt = Personas25_data_rev.copy()

In [87]:
Personas24_dt.columns

Index(['Idaccidente', 'Fecha', 'Mes', 'Hora', 'Comuna', 'Nomcomuna', 'Región',
       'Calidad', 'Sexo', 'Edad', 'Resultado'],
      dtype='object')

In [88]:
Personas24_dt = Personas24_dt.drop(columns=['Comuna'])

In [89]:
Personas25_dt.columns

Index(['Idaccidente', 'Fecha', 'Mes', 'Hora', 'Nomcomuna', 'Región', 'Calidad',
       'Sexo', 'Edad', 'Resultado'],
      dtype='object')

In [90]:
personas_training = pd.concat([Personas24_dt, Personas25_dt], ignore_index=True)
personas_training.head()

,Idaccidente,Fecha,Mes,Hora,Nomcomuna,Región,Calidad,Sexo,Edad,Resultado
0,1586623,2024-01-01,enero,00:00:00,quirihue,De Ñuble,conductor,masculino,40.0,leve
1,1586623,2024-01-01,enero,00:00:00,quirihue,De Ñuble,pasajero,femenino,24.0,leve
2,1586623,2024-01-01,enero,00:00:00,quirihue,De Ñuble,conductor,masculino,23.0,leve
3,1583417,2024-01-01,enero,00:30:00,san felipe,De Valparaíso,conductor,masculino,35.0,ileso
4,1583417,2024-01-01,enero,00:30:00,san felipe,De Valparaíso,conductor,masculino,0.0,ileso


In [91]:
Full_data = personas_training.merge(accidentes_training, on='Idaccidente', suffixes=('_persona', '_accidente'))

In [ ]:
personas_training.to_excel(path + 'personas_training.xlsx', index=False)
accidentes_training.to_excel(path + 'accidentes_training.xlsx', index=False)